# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [16]:
import numpy as np 
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use. 

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text. 

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available. 

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [17]:
# Import necessary libraries
import pandas as pd
import requests
import io
import os

# Create a directory for the dataset if it doesn't exist
os.makedirs('./kaggle/input/bbc-news', exist_ok=True)

# Download BBC News dataset from a GitHub repository
url = "https://raw.githubusercontent.com/susanli2016/PyCon-Canada-2019-NLP-Tutorial/master/bbc-text.csv"
response = requests.get(url)

# Save the dataset locally
with open('./kaggle/input/bbc-news/bbc_news.csv', 'w', encoding='utf-8') as f:
    f.write(response.text)

# Now read the CSV file from the local path
news = pd.read_csv('./kaggle/input/bbc-news/bbc_news.csv')
MAX_NEWS = 1000
DOCUMENT="text"  # Column name in this dataset
TOPIC="category"  # Column name in this dataset

# Display the first few rows to verify
news.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [18]:
news["id"] = news.index
news.head()

,category,text,id
0,tech,tv future in the hands of viewers with home th...,0
1,business,worldcom boss left books alone former worldc...,1
2,sport,tigers wary of farrell gamble leicester say ...,2
3,sport,yeading face newcastle in fa cup premiership s...,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,4


In [19]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database. 

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior. 

In [20]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters 
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance. 

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that. 

In [21]:
chroma_client = chromadb.PersistentClient(path="./kaggle")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it. 

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [22]:
collection_name = "news_collection"
# Check if the collection exists before trying to delete it
if collection_name in chroma_client.list_collections():
    chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)

It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***. 
* In the **document** we store the big text, it's a different column in each Dataset. 
* In **metadatas**, we can informa a list of topics. 
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS. 


In [23]:

collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

In [24]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id512', 'id861', 'id617', 'id201', 'id535', 'id615', 'id271', 'id184', 'id492', 'id971']], 'embeddings': None, 'documents': [['digital guru floats sub-$100 pc nicholas negroponte  chairman and founder of mit s media labs  says he is developing a laptop pc that will go on sale for less than $100 (£53).  he told the bbc world service programme go digital he hoped it would become an education tool in developing countries. he said one laptop per child could be   very important to the development of not just that child but now the whole family  village and neighbourhood . he said the child could use the laptop like a text book. he described the device as a stripped down laptop  which would run a linux-based operating system   we have to get the display down to below $20  to do this we need to rear project the image rather than using an ordinary flat panel.   the second trick is to get rid of the fat   if you can skinny it down you can gain speed and the ability to use smaller pro

## Vector MAP

In [25]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [26]:

getado = collection.get(ids="id141", 
                       include=["documents", "embeddings"])


In [27]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([[-2.49315649e-02,  3.38042527e-02, -3.06575187e-02,
        -3.90807353e-02,  1.40233273e-02,  1.55304158e-02,
         1.69949476e-02,  5.39894588e-02,  1.41636496e-02,
         1.89541541e-02,  2.76822913e-02,  4.40177731e-02,
         1.08812749e-03, -6.10495955e-02,  3.20915729e-02,
         8.54964033e-02,  8.12698603e-02,  5.02734259e-02,
        -4.07153182e-02,  1.75708011e-02,  9.60872602e-03,
         1.87131464e-02, -2.60261772e-03, -8.58199596e-03,
        -3.23509909e-02, -3.04508246e-02, -9.76185054e-02,
        -6.59352019e-02, -2.57009957e-02, -7.16404384e-03,
         4.39454950e-02,  2.12780386e-02, -7.78646693e-02,
         3.67388465e-02,  2.93054599e-02, -9.34431106e-02,
        -1.16834161e-03,  3.20133045e-02, -1.79879088e-02,
        -2.22471301e-02,  4.42495607e-02, -1.59083419e-02,
        -7.03627989e-02, -5.95747586e-03, -5.98260835e-02,
        -7.22549018e-03, -7.44215678e-03,  5.35251349e-02,
        -1.99125148e-02,  1.85347814e-02, -2.29192097e-0

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents. 

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search. 


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models. 

We are importing: 
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification. 

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2. 

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)



KeyboardInterrupt: 

The next step is to initialize the pipeline using the objects created above. 

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [29]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user. 

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**. 

We only need to join the two parts together to create the prompt that we are going to send to the model. 

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part. 

In [30]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #digital guru floats sub-$100 pc nicholas negroponte  chairman and founder of mit s media labs  says he is developing a laptop pc that will go on sale for less than $100 (£53).  he told the bbc world service programme go digital he hoped it would become an education tool in developing countries. he said one laptop per child could be   very important to the development of not just that child but now the whole family  village and neighbourhood . he said the child could use the laptop like a text book. he described the device as a stripped down laptop  which would run a linux-based operating system   we have to get the display down to below $20  to do this we need to rear project the image rather than using an ordinary flat panel.   the second trick is to get rid of the fat   if you can skinny it down you can gain speed and the ability to use smaller processors and slower memory.  the device will probably be exported as a kit of parts to be assembled locally to keep cos

Now all that remains is to send the prompt to the model and wait for its response!


In [ ]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])